#방법1
  
###PDF 파일에서 텍스트를 추출하고, 추출된 텍스트를 기반으로 OpenAI의 GPT-3.5-turbo 모델을 사용하여 질문과 그에 대한 전문적인 답변 쌍을 생성


1. PDF를 로드하고 페이지 별로 텍스트를 추출
2. 추출된 텍스트를 여러 문장으로 나누어 다양한 질문을 생성
3. GPT-3.5-turbo 모델을 사용하여 전문적인 답변을 생성하고, 이를 JSON 형식에 맞춰서 저장
4. 임베딩, 벡터화 하지 않음(from langchain.chains import RetrievalQA
, from langchain.vectorstores import Chr)

##pip install

In [ ]:
pip install openai==0.28

In [ ]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 17.3 MB/s eta 0:00:00


In [ ]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 3.8 MB/s eta 0:00:00


In [ ]:
pip install python-dotenv

In [ ]:
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

##방법1-1 PDF 편집 (테스트)

In [ ]:
import openai
from langchain.document_loaders import PyPDFLoader
import json

# Set your OpenAI API key here
openai.api_key = 'key 입력'

def get_arg_answer(query, verbose=False):
    prompt = f"""
   You are an Teacher/Professor
    Your task is to provide exactly 7 question(s) for an upcoming quiz/examination.
    You are not to provide more or less than this number of questions.
    The question(s) should be diverse in nature across the document.

    Now I'll give you the text you need to create.
    QUESTION should be written in Korean.
    {query}
    """
    #if verbose:
        #print(f'Prompt: {prompt}')

    questions = query.split(" / ")
    return questions

def get_gen_answer(questions, verbose=False):
    answers = []
    for question in questions:
        prompt = f"""
You're an expert on ports, logistics, and ships, so when the question comes up, you're going to give an educated, expert response to the relevant regulations.

Answers must be 300 characters or less
Please condense your answer to the maximum character length.
Your answers must be written in Korean.
Answers must be in complete sentences.

Output 5 question and answer pairs.
Print the questions and answers in the following format: "Question" and "Answer" JSON.
    {{
    "Questions": "",
    "Answer": ""
    }},
    {{
    "QUESTION": "",
    "Answer": ""
    }}


        {question}
        """
        #if verbose:
            #print(f'Prompt for question "{question}": {prompt}')

        # Call OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You must also provide an answer to each question."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            #max_tokens=512,
            stop=None,
            n=1,
        )

        # API response handling
        answer = response['choices'][0]['message']['content']
        answers.append(answer.strip())

    return answers

def generate_qa_from_pdf(pdf_path, verbose=False):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text = " ".join([page.page_content for page in docs])

    # Divide sentences by '/' to generate questions
    questions = get_arg_answer(text, verbose=verbose)

    # Generate expert answers for each question
    answers = get_gen_answer(questions, verbose=verbose)

    return questions, answers

# PDF file path
pdf_path = "/content/가이드북_편집.pdf"
questions, answers = generate_qa_from_pdf(pdf_path, verbose=True)


# Create Q&A pairs
qa_pairs = [{"QUESTION": q, "ANSWER": a} for q, a in zip(questions, answers)]

# Q&A 쌍을 순회하면서 출력
for idx, qa in enumerate(qa_pairs, start=1):
    print(f"{qa['ANSWER']}\n")

{
    "Questions": "외항선 입항신고와 승무원/승객명부에 대해 설명해주세요.",
    "Answer": "외항선 입항신고는 무역항의 수상구역으로 선박이 입항할 때 필요한 신고이며, 승무원/승객명부는 선박에 탑승한 인적사항을 신고하는 것입니다."
},
{
    "Questions": "강제도선 면제신청과 예선사용 면제신청은 무엇인가요?",
    "Answer": "강제도선 면제신청은 선장과 선박이 일정 횟수 이상 승선한 경우의 면제 신청이며, 예선사용 면제신청은 동일항만에 일정 횟수 이상 예선을 사용한 경우의 면제 신청입니다."
},
{
    "Questions": "선박평형수 신고와 선박보안정보통보 신고에 대해 설명해주세요.",
    "Answer": "선박평형수 신고는 선박의 평형수 처리 및 관련 사항을 신고하는 것이며, 선박보안정보통보 신고는 국제선박의 보안에 관한 사항을 신고하는 것입니다."
},
{
    "Questions": "저속운항선박지원신청은 무엇을 의미하나요?",
    "Answer": "저속운항선박지원신청은 특정 지역에서 저속으로 운항하는 선박이 지원을 받기 위해 해당 항구 관할구역에서 신청하는 것을 말합니다."
},
{
    "Questions": "외항선 입항신고와 승무원/승객명부의 신고처는 어디인가요?",
    "Answer": "외항선 입항신고와 승무원/승객명부 신고는 해양수산부의 PORT-MIS 또는 KL-NET, KTNET을 통해 신고할 수 있습니다."
}



##방법1-2 PDF 원본(포트미스 가이드북)

In [ ]:
import openai
from langchain.document_loaders import PyPDFLoader
import json

# Set your OpenAI API key here
openai.api_key = 'key 입력'

def get_arg_answer(query, verbose=False):
    prompt = f"""
   You are an Teacher/Professor
    Your task is to provide exactly 7 question(s) for an upcoming quiz/examination.
    You are not to provide more or less than this number of questions.
    The question(s) should be diverse in nature across the document.

    Now I'll give you the text you need to create.
    QUESTION should be written in Korean.
    {query}
    """
    #if verbose:
        #print(f'Prompt: {prompt}')

    questions = query.split(" / ")
    return questions

def get_gen_answer(questions, verbose=False):
    answers = []
    for question in questions:
        prompt = f"""
You're an expert on ports, logistics, and ships, so when the question comes up, you're going to give an educated, expert response to the relevant regulations.

Answers must be 300 characters or less
Please condense your answer to the maximum character length.
Your answers must be written in Korean.
Answers must be in complete sentences.
Print the questions and answers in the following format: "Question" and "Answer" JSON.
    {{
    "Questions": "",
    "Answer": ""
    }},
    {{
    "QUESTION": "",
    "Answer": ""
    }}


        {question}
        """
        #if verbose:
            #print(f'Prompt for question "{question}": {prompt}')

        # Call OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You must also provide an answer to each question."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            #max_tokens=512,
            stop=None,
            n=1,
        )

        # API response handling
        answer = response['choices'][0]['message']['content']
        answers.append(answer.strip())

    return answers

def generate_qa_from_pdf(pdf_path, verbose=False):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text = " ".join([page.page_content for page in docs])

    # Divide sentences by '/' to generate questions
    questions = get_arg_answer(text, verbose=verbose)

    # Generate expert answers for each question
    answers = get_gen_answer(questions, verbose=verbose)

    return questions, answers

# PDF file path
pdf_path = "/content/(2024)포트미스+가이드북.pdf"
questions, answers = generate_qa_from_pdf(pdf_path, verbose=True)


# Create Q&A pairs
qa_pairs = [{"QUESTION": q, "ANSWER": a} for q, a in zip(questions, answers)]

# Q&A 쌍을 순회하면서 출력
for idx, qa in enumerate(qa_pairs, start=1):
    print(f"{qa['ANSWER']}\n")

{
    "Question": "외항선 민원신고 절차",
    "Answer": "포트미스를 통해 외항선 민원신고를 접수하고 처리할 수 있으며, 해당 절차에는 외항입항신고(최초)가 포함됩니다."
}

{
    "Question": "승무원이란 무엇입니까?",
    "Answer": "승무원은 배나 선박에서 일하는 직원들을 의미합니다."
}

{
    "Questions": "항만시설 사용허가 신청서 어떻게 작성해야 하나요?",
    "Answer": "항만시설 사용허가 신청서는 해당 항만 관리기관에서 지정된 양식에 따라 작성하여야 합니다."
}

{
    "Question": "승무원",
    "Answer": "승무원의 근무시간과 휴식시간은 국제노동기구(ILO)의 규정 및 해당 국가의 노동법에 따라 규제됩니다."
}

{
    "Question": "항만시설 사용 허가신고를 위한 담당자는 누구입니까?",
    "Answer": "항만시설 사용 허가신고를 위한 담당자는 해당 항만의 수리 담당자입니다."
}

{
    "Question": "해운 및 항만 업무에서 가장 중요한 규정은 무엇인가요?",
    "Answer": "해운 및 항만 업무에서 가장 중요한 규정은 국제해사기구(IMO)의 규정입니다. IMO의 규정을 준수하여 안전하고 효율적인 운송을 유지해야 합니다."
}

{
    "Questions": "항만시설 사용허가 신청 방법은 무엇입니까?",
    "Answer": "항만시설 사용허가는 항만시설사용허가 신청서를 작성하여 담당자에게 제출해야 합니다."
},
{
    "Questions": "화물선의 내항출항 신고를 어디서 할 수 있습니까?",
    "Answer": "화물선의 내항출항 신고는 담당자(승인)에게 최초로 제출해야 합니다."
}

{
    "Question": "항만 안전을 유지하기 위한 중요한 규정은 무엇입니까?",
    "Answer": "항만 안전을 유지하기 위한 중요한 규정은 ISPS 규정입니다."
}
